npy -> {import numpy as np
       {np.load
       
npz -> {import scipy.sparse as sp
       {sp.load_npz

Remark : problem in load_data.
         add : n_test = X_test.shape[0]
         modify in X.test hstack : replace n by n_test

TP 07/01/2022 : SD-TSIA 211 - Optimization for machine learning - NLP

In [48]:
import numpy as np
import scipy.sparse as sp
from scipy.linalg import norm, solve
from scipy.optimize import check_grad
import matplotlib.pyplot as plt
import time

def load_data(file_name_matrix='tfidf_matrix_97MB.npz', file_name_feature_names='feature_names_97MB.npy',
	      file_name_labels='train_labels.npy', samples_in_train_set=10000,
	      samples_in_test_set=137562):
	# Recuperation des donnees
	TF_IDF_matrix = sp.load_npz(file_name_matrix)
	TF_IDF_feature_names = np.load(file_name_feature_names)
	train_labels = np.load(file_name_labels, allow_pickle=True)
	train_labels_numeric = ((train_labels == 'Oui') + 0)

	X = TF_IDF_matrix[:samples_in_train_set].toarray()
	y = train_labels_numeric[:samples_in_train_set] * 2 - 1

	X_test = TF_IDF_matrix[samples_in_train_set:samples_in_train_set+samples_in_test_set].toarray()
	y_test = train_labels_numeric[samples_in_train_set:samples_in_train_set+samples_in_test_set] * 2 - 1


	# Standardisation des données
	std_X = np.maximum(np.std(X, axis=0), 1e-7)
	X = X / std_X
	X_test = X_test / std_X

	n = X.shape[0]
	n_test = X_test.shape[0]
	m = X.shape[1]

	# Ajout d'une colonne de uns
	eX = np.hstack((np.ones((n,1)), X))
	eX_test = np.hstack((np.ones((n_test,1)), X_test))

	return eX, y, eX_test, y_test

Q3.1/ See proof on paper

Q3.2/ Code

In [50]:
#Data load
eX,y,eX_test,y_test = load_data()
n = np.size(eX_test[1])
rho = 1/n

#A function computing the value of f1 :
def f1(w0,w):
    res = 0
    for i in range(n):
        res+=m.log(1+m.exp(-y_test[i]*(np.dot(eX_test[i],w)+w0)))
    return res/n + rho*np.dot(w,w)/2

def grad_f1(w0,w):
    tmp_down = np.ones((1,n))/n
    tmp_up = np.zeros((1,1))
    for i in range(n):
        expp = m.exp(-y_test[i]*(np.dot(eX_test[i],w)+w0))
        tmp_down += eX_test[i]*(-y_test[i]/(1+expp)*(expp))
        tmp_up -= np.array([y_test[i]*expp])
        #Pour que tout se passe bien, xi doit être un nparray. Idem w
    return np.concatenate((tmp_up, tmp_down), axis=0)

w_test = np.ones((n,1))
grad_f1(0,w_test)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 576

In [34]:
#a = np.array([[1],[2]],[[1],[2]])
#b = np.array([[3],[4]],[[0],[0]])
p=3
c = np.ones((1,p))/p
c = c.reshape(p,1)
c

array([[0.33333333],
       [0.33333333],
       [0.33333333]])